In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import ast
import torch
import os
import json


In [2]:
from src.util_func import load_yaml,z_score_normalize,get_top_k_indices,run_command,round_down_to_power_of_two,load_yaml_args,normalize_to_neg1_pos1


In [3]:
from src.util_func import load_yaml,z_score_normalize,get_top_k_indices,run_command,round_down_to_power_of_two,load_yaml_args,normalize_to_neg1_pos1
from src.evaluation import evaluation
evaluator = evaluation()

In [24]:
output_file_name = 'output/Ablation/CTA/WebTable/mistral-7B-w-main-batch-size-4.csv'
output_file = pd.read_csv(output_file_name,index_col=0)
metrics = evaluator.process('CTA','WebTable',output_file)
metrics

Micro Score: 0.938132427843803

Macro Score: 0.7767259106717945


{'micro_f1': 0.938132427843803, 'macro_f1': 0.7767259106717945}

In [23]:
np.load('eval_result/time_dict_CTA_WebTable_0,1,2,3_batch-size-4.npy',allow_pickle=True).item()

{'init_select': 27.753063917160034,
 'init-train': 28.262178421020508,
 'batch-division': 50.33584022521973,
 'gradient-calculation': 430.92104387283325,
 'IF-Score': 516.7821741104126,
 'Final Selection': 517.067054271698}

{'init_select': 26.36381244659424,
 'init-train': 26.608953952789307,
 'batch-division': 45.89527630805969,
 'gradient-calculation': 488.21150374412537,
 'IF-Score': 528.6868059635162,
 'Final Selection': 528.9380893707275} 12
 {'init_select': 27.753063917160034,
 'init-train': 28.262178421020508,
 'batch-division': 50.33584022521973,
 'gradient-calculation': 430.92104387283325,
 'IF-Score': 516.7821741104126,
 'Final Selection': 517.067054271698} 4

## Runtime for batch=1 IF calculation
CUDA_VISIBLE_DEVICES=3 python cal_IF_single.py --yaml_path script/config_CTA_WebTable.yaml --process_num 1 --total_process_num 1

In [6]:
task = 'CTA'
dataset = 'WebTable'
count_val = 0
val_grad_dict = {}
for i in range(1,5,1):
    val_grad_dict_part = torch.load('grad/{}/{}/val_grad_{}.pkl'.format(task,dataset,i))
    for key in val_grad_dict_part.keys():
        val_grad_dict[key+count_val] = val_grad_dict_part[key]
    count_val += len(val_grad_dict_part)

In [1]:
import numpy as np
from sklearn.cluster import  KMeans
from sklearn.manifold import TSNE
import torch
import argparse
import json
import os
from tqdm import tqdm
import pandas as pd
# from FlagEmbedding import FlagModel
import time
# import submodlib
# from submodlib.functions.facilityLocation import FacilityLocationFunction
from datasets import Dataset
from src.load_dataset import ReTaskEvaluator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

import yaml
import argparse
from src.evaluation import evaluation
evaluator = evaluation()
import torch.multiprocessing as mp
import subprocess
from src.util_func import load_yaml,z_score_normalize,get_top_k_indices,run_command,round_down_to_power_of_two,load_yaml_args,normalize_to_neg1_pos1



In [ ]:
import pandas as pd
pd.read_json('/home/wys/DataSelection-IF/train/RE/RE/train-select-w-main-batch-size-4.json')

### Vary Selection Size

### Main Method MELD-DS

In [6]:
def z_score_normalize(sample_IF):
    """
    对sample_IF字典中各个'method'对应的值（键为index、值为float的字典）进行Z-score归一化。

    参数:
    sample_IF (dict): 包含多个'method'键的字典，每个'method'键对应的值为需要进行归一化处理的字典数据。

    返回:
    dict: 归一化后的字典，结构与输入的sample_IF一致，其中每个'method'键对应的值都已经完成Z-score归一化。
    """
    for method in sample_IF.keys():
        # 获取当前method对应需要归一化的值列表，保持原有顺序
        values_list = list(sample_IF[method].values())
        # 将列表转换为torch.Tensor
        tensor_value = torch.tensor(values_list).unsqueeze(1)  # 添加维度，变为二维张量

        # 计算均值和标准差
        mean_value = tensor_value.mean()
        std_value = tensor_value.std()

        # 进行Z-score归一化
        normalized_tensor = (tensor_value - mean_value) / std_value

        # 将归一化后的结果再转换回列表
        normalized_list = normalized_tensor.squeeze(1).tolist()

        # 更新原字典中当前method对应的值
        index_list = list(sample_IF[method].keys())
        normalized_dict = {}
        for index, normalized_value in zip(index_list, normalized_list):
            normalized_dict[index] = normalized_value

        sample_IF[method] = normalized_dict

    return sample_IF

In [57]:
## reverse total score
import json
import numpy as np
import pandas as pd
import torch
from src.util_func import load_yaml
task_list = [
    # ['RE','RE'],
    # ['ER','abt-buy'],
    # ['ER','semi-text-w'],
    # ['DC','hospital'],
    # ['DC','beer'],
    # ['DC','rayyan'],
    # ['DI','amazon'],
    # ['DI','walmart'],
    # ['AVE','oa_mine'],
    ['CTA','SimTab'],
    # ['CTA','WebTable'],
    # ['ER','walmart-amazon'],
    # ['ER','semi-text-c'],
    # ['ER','amazon-google'],
    # ['ER','wdc'],
    # ['SM','CMS']
]
for task,dataset in task_list:
    yaml_path = f'script/config_{task}_{dataset}.yaml'
    config = load_yaml(yaml_path)
    cluster_num = config['cluster_num']
    cluster_per_budget = config['sample_per_cluster']
    cluster_per_budget = 14
    train_file_path = config['train_file_path']
    train_file = pd.read_json(train_file_path)

    cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
    total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)
    sample_IF = torch.load(f'Influence/{task}/{dataset}/score.pkl',weights_only=False)
    sample_IF = z_score_normalize(sample_IF)

    ppl_array = np.zeros(len(total_score))
    for process_num in range(1,9,1): ## maximum of k process
        if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
            ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
            for index,row in ppl_df.iterrows():
                ppl_array[index] = row[0]

    # QuRating_Score = np.load(f'QuRating/data/select_index_main_writing/{task}-{dataset}-QuRating-score.npy',allow_pickle=True).item()
    # for key in range(len(total_score)):
    #     # total_score[i] = total_score[i] - ppl_array[i]
    #     if sample_IF['iterative'].__contains__(key):
    #         total_score[key] += sample_IF['iterative'][key]
    #     # total_score[i] = total_score[i] + 6 * QuRating_Score[i]
    for key in sample_IF['iterative'].keys():
        total_score[key] += sample_IF['iterative'][key]


    p2_choose_index = []
    count = 0
    for i in cluster_rank.keys():
        index_all = cluster_rank[i]
        sorted_score_all = total_score[index_all]
        sorted_index_all = index_all[np.argsort(-sorted_score_all)]
        # print(index_all,sorted_index_all)
        # break
        change = cluster_per_budget - len(set(index_all[:cluster_per_budget]).intersection(set(sorted_index_all[:cluster_per_budget])))
        count += change
        p2_choose_index.extend(sorted_index_all[:cluster_per_budget])
    select_IF_df = train_file.iloc[p2_choose_index]
    print(len(select_IF_df) / len(train_file))
    json.dump(select_IF_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-size-20.json'.format(task,dataset), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_100015/834544746.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/834544746.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/834544746.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/834544746.py:44: FutureWarning: Series.__getitem__ treating keys as positions is 

0.2166745171926519


/tmp/ipykernel_100015/834544746.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]


In [16]:
python Ablation_Study.py --yaml_path script/config_RE_RE.yaml --device 0,1 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

0.2066997145861499

In [ ]:
python Ablation_Study.py --yaml_path script/config_RE_RE.yaml --device 0,1 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var QuRating-size-10 QuRating-size-20 LESS-size-10 LESS-size-20

In [ ]:
python Ablation_Study.py --yaml_path script/config_ER_amazon-google.yaml --device 2,3 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

In [ ]:
python Ablation_Study.py --yaml_path script/config_CTA_SimTab.yaml --device 0,1 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20 QuRating-size-10 QuRating-size-20 LESS-size-10 LESS-size-20

In [ ]:
python Ablation_Study.py --yaml_path script/config_ER_amazon-google.yaml --device 2,3 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var QuRating-size-10 QuRating-size-20 LESS-size-10 LESS-size-20

In [ ]:
python Ablation_Study.py --yaml_path script/config_CTA_SimTab.yaml --device 2,3 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var main-size-20 main-size-10

### DataInf

In [ ]:
IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

In [ ]:
QuRating-size-10 QuRating-size-20 LESS-size-10 LESS-size-20

In [11]:
task = 'CTA'
dataset = 'SimTab'
percentage = 20
select_num_df = pd.read_json(f'train/{task}/{dataset}/train-select-w-main-size-{percentage}.json')
select_num = len(select_num_df)
yaml_path = f'script/config_{task}_{dataset}.yaml'
config = load_yaml(yaml_path)
train_file_path = config['train_file_path']
train_file = pd.read_json(train_file_path)
if os.path.exists('Influence_single/{}/{}'.format(task,dataset)):
    sample_IF_single = torch.load('Influence_single/{}/{}/score.pkl'.format(task,dataset),weights_only=False)
    for IF_method in sample_IF_single.keys():
        select_IF_single = get_top_k_indices(sample_IF_single[IF_method],k=select_num,IF=True)
        select_IF_single_df = train_file.iloc[select_IF_single]
        json.dump(select_IF_single_df.to_dict(orient='records'), open(f'train/{task}/{dataset}/train-select-w-IF-single-size-{percentage}.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
        print(len(select_IF_single_df))

1380
1380
1380


## DSIR

### SuperFilter

In [13]:
task = 'CTA'
dataset = 'SimTab'
percentage = 20
select_num_df = pd.read_json(f'train/{task}/{dataset}/train-select-w-main-size-{percentage}.json')
select_size = len(select_num_df)
step_3_command = f'CUDA_VISIBLE_DEVICES=3 python code_ifd/select_data.py \
--json_data_path output/{task}-{dataset}-gpt2.jsonl \
--json_save_path ../DataSelection-IF/train/{task}/{dataset}/train-select-w-SuperFiltering-size-{percentage}.json \
--sample_num {select_size} '
step_3_command

'CUDA_VISIBLE_DEVICES=3 python code_ifd/select_data.py --json_data_path output/CTA-SimTab-gpt2.jsonl --json_save_path ../DataSelection-IF/train/CTA/SimTab/train-select-w-SuperFiltering-size-20.json --sample_num 1380 '

## QuRating

In [39]:
python QuRating_Selection.py --task CTA --dataset SimTab --device 0 --model_path ../../model/QuRating-1.3B --select_size 1300

SyntaxError: invalid syntax (1708660588.py, line 1)

In [16]:
task = 'CTA'
dataset = 'SimTab'
select_file_size = 1300
select_QuRating = np.load('QuRating/data/select_index_main_expert/{}-{}-QuRating-size-{}.npy'.format(task,dataset,select_file_size))
yaml_path = f'script/config_{task}_{dataset}.yaml'
config = load_yaml(yaml_path)
train_file_path = config['train_file_path']
train_file = pd.read_json(train_file_path)
select_QuRating_df = train_file.iloc[select_QuRating]
json.dump(select_QuRating_df.to_dict(orient='records'), open(f'train/{task}/{dataset}/train-select-w-QuRating-size-20.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

## LESS

In [19]:
task = 'CTA'
dataset = 'SimTab'
file = pd.read_json(f'../LESS/LESS_output/{task}_{dataset}/LESS_subset.jsonl',lines=True)
def content_extraction(row):
    instruction = row[-1][0]['content']
    input = ''
    output = row[-1][1]['content']
    return instruction,input,output
content_subset = file.apply(content_extraction,axis=1,result_type='expand')
content_subset.columns = ['instruction','input','output']
json.dump(content_subset.iloc[:,:1300].to_dict(orient='records'), open(f'../DataSelection-IF/train/{task}/{dataset}/train-select-w-LESS-size-20.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_37821/345711980.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  instruction = row[-1][0]['content']
/tmp/ipykernel_37821/345711980.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output = row[-1][1]['content']


## Vary Batch Size

In [ ]:
task = 'RE'
dataset = 'RE'
batch_size = 4
sample_IF = torch.load('Influence/{}/{}/batch-size-{}/score.pkl'.format(task,dataset,batch_size),weights_only=False)
sample_IF = z_score_normalize(sample_IF)



## FL Score
greedyList_All_norm = z_score_normalize(greedyList_All)
## Flatten
greedyList_All_norm_flatten = {}
for key in greedyList_All_norm.keys():
    for index in greedyList_All_norm[key]:
        greedyList_All_norm_flatten[index] = greedyList_All_norm[key][index]

total_score = np.zeros(len(train_file))
## calculate ppl
# for index,row in ppl.iterrows():
#     total_score[index] += row[0]

for index in range(len(total_score)):
    total_score[index] = ppl_array[index]
## add FL
for key in greedyList_All_norm_flatten.keys():
    total_score[key] += greedyList_All_norm_flatten[key]
## add global_IF
for key in sample_IF['iterative']:
    total_score[key] -= sample_IF['iterative'][key] ## Influence Score与performance成反比，所以是-=
    
## 遍历cluster,排序
cluster_rank = {}
for i in range(cluster_num): ## 100 is the cluster number, hyper-parameter
    cluster_rank[i] = {}
    cluster_index = cluster_indices[i]
    sorted_index = cluster_index[np.argsort(-total_score[cluster_index])] ## return to global index
    cluster_rank[i] = sorted_index

p2_choose_index = []
cluster_per_budget = sample_per_cluster
for i in range(cluster_num):
    p2_choose_index.extend(cluster_rank[i][:cluster_per_budget])

## save FL Score for ablation
create_folder_for_file('selection/{}/{}/batch-size-{}/FL-Score.pkl'.format(args.task,args.dataset))
torch.save(greedyList_All_norm_flatten,'selection/{}/{}/batch-size-{}/FL-Score.pkl'.format(args.task,args.dataset))
## save total score for ablation
torch.save(total_score,'selection/{}/{}/batch-size-{}/Total-Score.pkl'.format(args.task,args.dataset))
## save cluster division for ablation and furthur experiment
torch.save(cluster_rank,'selection/{}/{}/batch-size-{}/Cluster-Rank.pkl'.format(args.task,args.dataset))

## output

selected_df = train_file.iloc[p2_choose_index]

if(task=='SM'):
    train_file_pos = train_file[train_file['output']=="{'Output': 'match'}"]
    selected_df = pd.concat([selected_df,train_file_pos])
    print('add positive samples')

json.dump(selected_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-batch-size-{}.json'.format(args.task,args.dataset,batch_size), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)